In [26]:
import numpy as np
import matplotlib.pyplot as plt
from aif360.sklearn.metrics import average_odds_difference
from aif360.sklearn.metrics import equal_opportunity_difference
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.core.problem import ElementwiseProblem
from sklearn.metrics import f1_score, confusion_matrix, make_scorer
from fairlearn.metrics import (
    MetricFrame,
    count,
    selection_rate,
    equalized_odds_difference,
    false_positive_rate,
    false_negative_rate,
    demographic_parity_difference,
    true_positive_rate,
    true_negative_rate
)

from fairlearn.datasets import fetch_adult
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.termination import get_termination
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.crossover.hux import HalfUniformCrossover
from pymoo.visualization.scatter import Scatter


import dill


In [3]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

(X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
    X_raw, y, A, test_size=0.8, random_state=12345, stratify=y
)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)


numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)


/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [29]:
metrics = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "recall":recall_score,
    "f1 score": f1_score,
    "selection rate": selection_rate,
    "false positive rate": false_positive_rate,
    "true positive rate": true_positive_rate,
    "false negative rate": false_negative_rate,
    "true negative rate": true_negative_rate,
    "count": count,
}
metric_frame = MetricFrame(
    metrics=metrics, y_true=y_test, y_pred=y_pred, sensitive_features=A_test
)

In [37]:
#MetricFrame(metrics=true_positive_rate, y_true=y_test, y_pred=y_pred, sensitive_features=A_test).difference()

In [30]:
metric_frame.by_group

,accuracy,precision,recall,f1 score,selection rate,false positive rate,true positive rate,false negative rate,true negative rate,count
sex,,,,,,,,,,
Female,0.933566,0.764133,0.560400,0.646598,0.079535,0.021042,0.560400,0.439600,0.978958,12900.0
Male,0.829487,0.746467,0.664319,0.703001,0.270345,0.098447,0.664319,0.335681,0.901553,26174.0


In [12]:
metric_frame.by_group.diff().abs()

,selection rate,accuracy,precision,recall,false positive rate,true positive rate,count
sex,,,,,,,
Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Male,0.19081,0.104079,0.017666,0.103919,0.077405,0.103919,13274.0


In [19]:
# Statistical parity difference
metric_frame.by_group['selection rate'].diff().abs()

sex
Female        NaN
Male      0.19081
Name: selection rate, dtype: float64

In [23]:
# Disparate Impact
metric_frame.by_group['selection rate'].min()/metric_frame.by_group['selection rate'].max()

0.29419814111244036

In [16]:
# Equal Opportunity Difference
metric_frame.by_group[['true positive rate']].diff().abs()

,true positive rate
sex,
Female,NaN
Male,0.103919


In [18]:
# Predictive Equality
#https://www.nature.com/articles/s41598-022-07939-1
metric_frame.by_group[['false positive rate']].diff().abs()

,false positive rate
sex,
Female,NaN
Male,0.077405


In [15]:
# Average Odds Difference
metric_frame.by_group[['false positive rate','true positive rate']].diff().abs().mean(axis = 1)

sex
Female         NaN
Male      0.090662
dtype: float64

In [8]:
average_odds_difference(y_test,y_pred, prot_attr=A_test, priv_group='Male',pos_label=1)

-0.09066201831264031

In [7]:
equal_opportunity_difference(y_test,y_pred, prot_attr=A_test, priv_group='Male',pos_label=1)

-0.10391866767223013